In [71]:
!pip install phidata groq


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import os
from dotenv import load_dotenv
import smtplib
from email.message import EmailMessage

# Load environment variables from .env
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

In [25]:
from phi.model.groq import Groq

recipe_scout = Groq(
    model="llama-3.1-70b-versatile",
    api_key=GROQ_API_KEY
)

In [26]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.4,
    max_tokens=2048
)


In [28]:
from phi.agent import Agent
from phi.model.groq import Groq

# Make sure your API key is set
import os
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Create the LLM model
llm = Groq(
    model="llama-3.1-70b-versatile",
    api_key=GROQ_API_KEY,
    max_tokens=2048
)

# Now pass the LLM instance to Agent
recipe_scout = Agent(
    name="Recipe Scout",
    role="Nutrition Researcher",
    goal="Find low-GI, high-fiber recipes for blood sugar stability",
    instructions=[
        "Search globally inspired recipes",
        "Focus on low glycemic index foods",
        "Explain why each recipe is good for glucose control",
        "Return exactly 10 recipes"
    ],
    model=llm  # <-- make sure this is the Groq instance, not a string or dict
)



In [29]:
clinical_reviewer = Agent(
    name="Clinical Reviewer",
    role="Clinical Nutrition Analyst",
    goal="Evaluate recipes for blood sugar safety",
    instructions=[
        "Identify hidden sugars, refined carbs, sodium risks",
        "Rank top 3 safest recipes",
        "Add caution notes for risky items",
        "Use medical-style reasoning"
    ],
    model=llm
)


In [30]:
mailer_agent = Agent(
    name="Mailer",
    role="Automation Agent",
    goal="Send the clinical review via email",
    instructions=[
        "Format response as a professional email",
        "Do not change content",
        "Send using configured SMTP"
    ],
    model=llm
)


In [32]:
import os
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

# --- Email function ---
def send_email(body):
    msg = EmailMessage()
    msg["Subject"] = "Clinical Review: Blood Sugar Friendly Recipes"
    msg["From"] = EMAIL_USER
    msg["To"] = "janaa7223@gmail.com"  # recipient
    msg.set_content(body)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_USER, EMAIL_APP_PASSWORD)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

# --- Multi-agent workflow ---
try:
    # Agent 1: Recipe Scout
    scout_output = recipe_scout.run("List 10 low GI recipes for blood sugar stability")

    # Agent 2: Clinical Reviewer
    review_output = clinical_reviewer.run(f"Review the following recipes:\n{scout_output}")

    # Agent 3: Mailer (just call send_email directly)
    send_email(review_output)

except Exception as e:
    print("Workflow failed:", e)


Workflow failed: Error code: 400 - {'error': {'message': 'The model `llama3-groq-70b-8192-tool-use-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


In [54]:
import os
from dotenv import load_dotenv
import smtplib
from email.message import EmailMessage

# Load environment variables
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_APP_PASSWORD = os.getenv("EMAIL_APP_PASSWORD")

In [60]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="groq/llama3-70b-8192",
    temperature=0.4,
    max_tokens=2048
)


In [59]:
from phi.agent import Agent

recipe_scout = Agent(
    name="Recipe Scout",
    role="Nutrition Researcher",
    goal="Find low-GI, high-fiber recipes for blood sugar stability",
    instructions=[
        "Search globally inspired recipes",
        "Focus on low glycemic index foods",
        "Explain why each recipe is good for glucose control",
        "Return exactly 10 recipes"
    ],
    model=llm
)

ValidationError: 1 validation error for Agent
model
  Input should be a valid dictionary or instance of Model [type=model_type, input_value='groq/llama3-70b-8192', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [45]:
clinical_reviewer = Agent(
    name="Clinical Reviewer",
    role="Clinical Nutrition Analyst",
    goal="Evaluate recipes for blood sugar safety",
    instructions=[
        "Identify hidden sugars, refined carbs, sodium risks",
        "Rank top 3 safest recipes",
        "Add caution notes for risky items",
        "Use medical-style reasoning"
    ],
    model=llm
)

In [50]:
mailer_agent = Agent(
    name="Mailer",
    role="Automation Agent",
    goal="Send the clinical review via email",
    instructions=[
        "Format response as a professional email",
        "Do not change content",
        "Send using configured SMTP"
    ],
    model=llm
)


In [51]:
def send_email(body):
    msg = EmailMessage()
    msg["Subject"] = "Clinical Review: Blood Sugar Friendly Recipes"
    msg["From"] = EMAIL_USER
    msg["To"] = "janaa7223@gmail.com"
    msg.set_content(body)

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_USER, EMAIL_APP_PASSWORD)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print("Failed to send email:", e)

# --- Run Workflow ---
try:
    scout_output = recipe_scout.run("List 10 low GI recipes for blood sugar stability")
    review_output = clinical_reviewer.run(f"Review the following recipes:\n{scout_output}")
    send_email(review_output)
except Exception as e:
    print("Workflow failed:", e)

Workflow failed: Error code: 400 - {'error': {'message': 'The model `llama3-groq-70b-8192-tool-use-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}


In [75]:
from dotenv import load_dotenv
load_dotenv()

import os
from phi.agent import Agent
from phi.model.groq import Groq

# Optional sanity check
print(os.getenv("GROQ_API_KEY"))

# Shared LLM config
llm = Groq(
    model="llama3-70b-8192",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.4
)

# -------- Agent 1: Recipe Scout --------
recipe_scout = Agent(
    name="Recipe Scout",
    role="Nutrition Researcher",
    goal="Find recipes that stabilize blood sugar",
    description="You are a nutrition research expert.",
    model=llm,
    show_tool_calls=True
)

# -------- Agent 2: Clinical Reviewer --------
clinical_reviewer = Agent(
    name="Clinical Reviewer",
    role="Clinical Nutritionist",
    goal="Clinically review recipes for glycemic safety",
    description="You are a clinical nutritionist.",
    model=llm,
    temperature=0.3
)

# -------- Workflow Orchestration --------

# Task 1
scout_output = recipe_scout.run(
    "List 10 low glycemic index recipes for blood sugar stability. "
    "Provide brief explanations."
)

# Task 2 (depends on Task 1)
review_output = clinical_reviewer.run(
    f"""
    Review the following recipes and rank the top 3 safest options.
    Provide clinical justification and caution notes.

    Recipes:
    {scout_output}
    """
)

print("\n--- FINAL OUTPUT ---\n")
print(review_output)


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


BadRequestError: Error code: 400 - {'error': {'message': 'The model `llama3-groq-70b-8192-tool-use-preview` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}